<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Library-and-Data-Import" data-toc-modified-id="Library-and-Data-Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Library and Data Import</a></span></li><li><span><a href="#Pre-processsing" data-toc-modified-id="Pre-processsing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pre-processsing</a></span></li><li><span><a href="#Extracting-cost-data-and-merging-with-the-applications/measures-database" data-toc-modified-id="Extracting-cost-data-and-merging-with-the-applications/measures-database-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extracting cost data and merging with the applications/measures database</a></span></li><li><span><a href="#Plotting" data-toc-modified-id="Plotting-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plotting</a></span></li><li><span><a href="#Notes" data-toc-modified-id="Notes-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Notes</a></span></li></ul></div>

Please proceed to the 'Plotting' section to find the interactive graphs by clicking on Plotting above.

# Library and Data Import

In [1]:
# Importing Python libraries necessary for the analysis
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pylab as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)
# from bs4 import BeautifulSoup # Needed to read the html Wikipedia page

In [3]:
london_auths = ['City of London','City of Westminster','Kensington and Chelsea','Hammersmith and Fulham',
                    'Wandsworth','Lambeth','Southwark','Tower Hamlets','Hackney','Islington','Camden',
                    'Brent','Ealing','Hounslow','Richmond','Kingston','Merton','Sutton','Croydon','Bromley',
                    'Lewisham','Greenwich','Bexley','Havering','Barking and Dagenham','Redbridge','Newham',
                    'Waltham Forest','Haringey','Enfield','Barnet','Harrow','Hillingdon']

In [4]:
# Reading GHG Data file as a whole
xls_data = pd.ExcelFile('Copy of GHG Data - 10022021 - Master.xlsx', engine='openpyxl')

# Reading each sheet in the excel file seperately. If names of sheets change this command won't work
df_app = pd.read_excel(xls_data, sheet_name='Application Level')
df_meas = pd.read_excel(xls_data, sheet_name='Measure Level')

In [5]:
# Reading Cost Data file as a whole
xls_costs = pd.ExcelFile('Copy of Cost data master copy.xlsx', engine='openpyxl')

# Reading the cost parameters sheet
df_costs = pd.read_excel(xls_costs, sheet_name='Cost Fraud Control Parameters', header=[23])

# Pre-processsing

We need to clean the data before we start doing our analysis. This includes preparing the two datasets for merging, merging them etc.

In [6]:
# Removing unnecessary columns and rows

# Dropping columns with 'Unnamed' in their name. These are unnecessary columns 
df_costs = df_costs[df_costs.columns.drop(list(df_costs.filter(regex='Unnamed')))]

# Dropping rows with less than 3 non-NaN values and rows that are the same as the column names
df_costs.dropna(thresh=3, inplace=True)
df_costs = df_costs[(df_costs != df_costs.columns).all(axis=1)]

# Converting Measure column to be capitalised as with the GHG data measure column
df_costs['Primary Measure - Insulation'] = df_costs['Primary Measure - Insulation'].str.capitalize()
df_meas['Measure_Name'] = df_meas['Measure_Name'].str.capitalize()

We merge the measures and applications sheets of the data to collect all the data in one dataframe

In [7]:
# Merging datasets based on the same Application ID
apps_meas = pd.merge(df_meas, df_app, on=['Application_ID'])

We extract the number of rooms where rooms is the unit for the measure

In [8]:
# Extracting number of rooms and checking if they are above or below 4
apps_meas['insulated_rooms'] = apps_meas.apply(lambda row:'Less than 4' if row['Quantity_Units'] == 'Number of Rooms' and int(row['Quantity_of_Measure']) < 4
                                                 else '4 or more' if row['Quantity_Units'] == 'Number of Rooms' and int(row['Quantity_of_Measure']) >= 4
                                                 else 'None', axis=1)

In [9]:
# Setting property type to any for retrofit measures that don't distinguish between property type (in the apps/measures dataframe)
apps_meas['Property_Type_x'] = apps_meas.apply(lambda row: 'Any' if row['Measure_Name'] not in ['External solid wall insulation','Internal solid wall insulation','Cavity wall insulation'] else row['Property_Type_x'], axis=1)

# Setting property type to any for retrofit measures that don't distinguish between property type
df_costs['Unit Type \n(Insulation Qualifier)'].fillna('Any', inplace=True)

In [10]:
# Setting all remaining property types to equivalent property types in cost sheet
apps_meas['Property_Type_x'] = apps_meas.apply(lambda row: 'Semi-Detached/Terrace/Bungalow' if row['Measure_Name']=='Cavity wall insulation' and row['Property_Type_x'] in ['Bungalow','Semi-Detached/Terrace','Terraced House','Semi-Detached House','Semi-Detached','Mid-Terrace','End-Terrace','Bungalow: Semi-Detached & End Terrace','Bungalow: Mid Terrace']
                                               else 'Semi-Detached/Terrace' if row['Property_Type_x'] in ['Terraced House','Semi-Detached House','Semi-Detached','Mid-Terrace','End-Terrace','Bungalow: Semi-Detached & End Terrace','Bungalow: Mid Terrace']\
                                               else 'Detached' if row['Property_Type_x'] in ['Detached House','House: Detached','Park Home','Bungalow: Detached']
                                               else 'Flat' if row['Property_Type_x'] in ['Flat: 3 External Walls','Flat: 2 External Walls'] 
                                               else row['Property_Type_x'], axis=1)

# Setting all remaining property types to equivalent property types in cost sheet
apps_meas['Measure_Name'] = apps_meas.apply(lambda row: 'Door' if row['Measure_Name'] in ['Energy efficient replacement doors']\
                                               else 'Heating controls (control)' if row['Measure_Name'] in ['Heating controls']\
                                               else 'Air source heat pump - average' if row['Measure_Name'] in ['Air source heat pump','Hybrid heat pumps']\
                                               else 'Douple glazing' if row['Measure_Name'] in ['Double/triple glazing']\
                                               else 'Room in roof insulation' if row['Measure_Name'] in ['Room-in-roof insulation']\
                                               else 'Biomass boilers' if row['Measure_Name'] in ['Biomass boiler']\
                                               else 'Hot water thermostat' if row['Measure_Name'] in ['Hot water tank thermostats']\
                                               else 'Ground source heat pumps - average' if row['Measure_Name'] in ['Ground source heat pump']\
                                               else 'Loft insulation (board/wool)' if row['Measure_Name'] in ['Loft insulation']\
                                               else row['Measure_Name'], axis=1)

# Extracting cost data and merging with the applications/measures database

In [11]:
# Merging the cost and apps/measures dataframes on measure name and property type
merged = pd.merge(apps_meas,df_costs, left_on=['Measure_Name','Property_Type_x'], right_on=['Primary Measure - Insulation','Unit Type \n(Insulation Qualifier)'], how='left')

In [13]:
# Picking the cost thresholds according to whether the Local authority is a London authority
merged['25%'] = merged.apply(lambda row: row['25% Below Avg Cost.1'] if row['Local_Authority'] in london_auths\
                                               else row['25% Below Avg Cost'], axis=1)

merged['125%'] = merged.apply(lambda row: row['25% Above Avg Cost.1'] if row['Local_Authority'] in london_auths\
                                               else row['25% Above Avg Cost'], axis=1)

merged['150%'] = merged.apply(lambda row: row['50% Above Avg Cost.1'] if row['Local_Authority'] in london_auths\
                                               else row['50% Above Avg Cost'], axis=1)

In [14]:
# Converting quantity of measure and quote amount columns to numerical to be able to calculate later on
cols = ['Quantity_of_Measure','Quote_Amount']
merged[cols] = merged[cols].apply(pd.to_numeric, errors='coerce')

# Calculating 25% and 125% of absolute cost 
merged.loc[:,'25%_cost'] = merged.loc[:,'Quantity_of_Measure']*merged.loc[:,'25%']
merged.loc[:,'125%_cost'] = merged.loc[:,'Quantity_of_Measure']*merged.loc[:,'125%']

# Defining status function to assess whether cost of measure is too high or low 
def status(row):
    if row['Quote_Amount'] < row['25%_cost']:
        val = 'Low'
    elif row['Quote_Amount'] > row['125%_cost']:
        val = 'High'
    else:
        val = 'Acceptable'
    return val

# Extractinng status
merged['Status'] = merged.apply(status, axis=1)

# Plotting

Plotting the shares of high, low and acceptable quotes.
- You can pick in **Group_by** whether you want to group quotes by installers or measures.
- **Cutoff** picks the cutoff point for amount of quotes a measure or installer can have to be included in the analysis. This is done to remove installers with only 1 or 2 quotes that are 'High' and thus 100% of their quotes will be high.
- **Sort_by** lets you sort by total number of quotes, and share (%) of high, low and acceptable quotes.
- **Top** picks the top X (default 25) installers or measures for the given sorting option.

In [48]:
sns.set_style('darkgrid')
@interact
def plot(Group_by = ['Installer_Company_Name','Measure_Name'], Cutoff=(1, 1000, 1),Sort_by=['Total',('Status','Acceptable'),('Status','High'),('Status','Low')], 
                            Top=(1, 50, 1), ):
    frame = pd.DataFrame(merged.groupby([Group_by])['Status'].value_counts(normalize=True)*100)
    frame = frame.unstack().sort_values([('Status', 'High')],ascending=False)
    frame['Total'] = merged.groupby([Group_by])['Status'].value_counts().unstack().sum(axis=1)
    
    frame = frame[frame['Total']>Cutoff].sort_values([Sort_by], ascending=False)[:Top]
    fig, ax = plt.subplots()
    frame.sort_values([Sort_by],)['Status'].plot.barh(figsize=(8,12), width=0.8, color=['g', 'r', '#FFA500'], ax=ax)
    ax.set_xlabel('Share of quote status (%)',fontsize=16)
    ax.set_ylabel('Installer',fontsize=16)
    for p in ax.patches:
        ax.annotate(str(p.get_width())[:4]+'%', (p.get_width()+2,p.get_y() * 1.01), fontsize=14)
        
    if Group_by == 'Measure_Name':
        groupby = 'measures'
        ax.set_ylabel('Measures',fontsize=16)
    else:
        groupby = 'installers'
        ax.set_ylabel('Installer',fontsize=16)
        
    if Sort_by == 'Total':
        ax.set_title(f'Top {Top} {groupby} by share of {Sort_by} status (Cutoff = {Cutoff})',fontsize=18)
    else:
        ax.set_title(f'Top {Top} {groupby} by share of {Sort_by[1]} status (Cutoff = {Cutoff})',fontsize=18)
    
    ax.tick_params(axis = 'both', which = 'major', labelsize = 18)
    return frame.round(2)

interactive(children=(Dropdown(description='Group_by', options=('Installer_Company_Name', 'Measure_Name'), val…

Plotting the volume of high, low and acceptable quotes.
- You can pick in **Group_by** whether you want to group quotes by installers or measures.
- **Cutoff** picks the cutoff point for amount of quotes a measure or installer can have to be included in the analysis. This is not as relevant for this plot as we are sorting by volume, not shares.
- **Sort_by** lets you sort by total number of quotes, and number of high, low and acceptable quotes.
- **Top** picks the top X (default 25) installers or measures for the given sorting option.

Plotting by volume of quotes:

In [49]:
sns.set_style('darkgrid')
@interact
def plot(Group_by = ['Installer_Company_Name','Measure_Name'], Cutoff=(1, 1000, 1),Sort_by=['Total',('Status','Acceptable'),('Status','High'),('Status','Low')], 
                            Top=(1, 50, 1), ):
    frame = pd.DataFrame(merged.groupby([Group_by])['Status'].value_counts())
    frame = frame.unstack().sort_values([('Status', 'High')],ascending=False)
    frame['Total'] = merged.groupby([Group_by])['Status'].value_counts().unstack().sum(axis=1)
    
    frame = frame[frame['Total']>Cutoff].sort_values([Sort_by], ascending=False)[:Top]
    fig, ax = plt.subplots()
    frame.sort_values([Sort_by],)['Status'].plot.barh(figsize=(8,12), width=0.8, color=['g', 'r', '#FFA500'], ax=ax)
    ax.set_xlabel('Share of quote status (%)',fontsize=16)
    ax.set_ylabel('Installer',fontsize=16)
    for p in ax.patches:
        ax.annotate(str(int(p.get_width())), (p.get_width()+2,p.get_y() * 1.01), fontsize=14)
        
    if Group_by == 'Measure_Name':
        groupby = 'measures'
        ax.set_ylabel('Measures',fontsize=16)
    else:
        groupby = 'installers'
        ax.set_ylabel('Installer',fontsize=16)
        
    if Sort_by == 'Total':
        ax.set_title(f'Top {Top} {groupby} by volume of {Sort_by} status quotes (Cutoff = {Cutoff})',fontsize=18)
    else:
        ax.set_title(f'Top {Top} {groupby} by volume of {Sort_by[1]} status quotes (Cutoff = {Cutoff})',fontsize=18)
    
    ax.tick_params(axis = 'both', which = 'major', labelsize = 18)
    return frame.round(2)

interactive(children=(Dropdown(description='Group_by', options=('Installer_Company_Name', 'Measure_Name'), val…

# Notes
1. Loft insulation has 2 types in the cost sheet but there's no information in the applications/measures sheet to distinguish between board and spray loft insulation measures. Currently applying all to board/wool.
2. Park homes in measures and applications but not in costs. Categorised as detached.
3. Extra semi-detached/terraced/bungalow unit type in costs sheet for cavity wall insulation measures
4. No way to distinguish between double and triple glazing measures in apps/measures sheet. Categorising all as double glazing.
5. No hybrid heat pumps in cost sheet. Currently assigning to air-source heat pump.